# A Macroscopically Estimating Model with Endogeneity Considered for Lane-mean Speeds and Speed Deviations (4lanes)

## 1 Structuring Data

### 1.1 Import Data

In [26]:
import numpy as np
import pandas as pd
import xlrd
from linearmodels import IV3SLS
import datetime
import math
import os
from dateutil.relativedelta import * # for incement of month

In [27]:
# filelist = []
# file_dir = 'G:\\Modeling Endogeneity\\Data'

# for root, dirs, files in os.walk(file_dir):
#     filelist = files

# file_200m_2lanes = [file for file in filelist if '200m_2lanes' in file]

In [28]:
data_up_fs = pd.DataFrame()
data_do_fs = pd.DataFrame()
data_up_o = pd.DataFrame()
data_up_tftp = pd.DataFrame()
data_up_vmtvht = pd.DataFrame()

# 12 for the number of month
for num in range(12):
    monnum = num + 1
    # flow,speed
    file_up_fs = 'G:\\Modeling Endogeneity\\Data\\400m_4lanes_up_fs_' + str(monnum) + '.xlsx'
    file_do_fs = 'G:\\Modeling Endogeneity\\Data\\400m_4lanes_do_fs_' + str(monnum) + '.xlsx'
    # occupancy
    file_up_o = 'G:\\Modeling Endogeneity\\Data\\400m_4lanes_up_o_' + str(monnum) + '.xlsx'
    file_do_o = 'G:\\Modeling Endogeneity\\Data\\400m_4lanes_do_o_' + str(monnum) + '.xlsx'
    # truck flow, truck percentage
    file_up_tftp = 'G:\\Modeling Endogeneity\\Data\\400m_4lanes_up_tftp_' + str(monnum) + '.xlsx'
    # file_do_tftp = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_tftp_1.xlsx'
    # truck vmt, trauck vht
    file_up_vmtvht = 'G:\\Modeling Endogeneity\\Data\\400m_4lanes_up_vmtvht_' + str(monnum) + '.xlsx'
    # file_do_vmtvht = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_vmtvht_1.xlsx'

    data_up_fs_t = pd.read_excel(file_up_fs)
    data_do_fs_t = pd.read_excel(file_do_fs)
    data_up_o_t = pd.read_excel(file_up_o)
    # data_do_o = pd.read_excel(file_do_o)
    data_up_tftp_t = pd.read_excel(file_up_tftp)
    # data_do_tftp = pd.read_excel(file_do_tftp)
    data_up_vmtvht_t = pd.read_excel(file_up_vmtvht)
    # data_do_vmtvht = pd.read_excel(file_do_vmtvht)
    data_up_fs = data_up_fs.append(data_up_fs_t, sort=True)
    data_do_fs = data_do_fs.append(data_do_fs_t, sort=True)
    data_up_o = data_up_o.append(data_up_o_t, sort=True)
    data_up_tftp = data_up_tftp.append(data_up_tftp_t, sort=True)
    data_up_vmtvht = data_up_vmtvht.append(data_up_vmtvht_t, sort=True)

In [29]:
data_up_vmtvht.describe() # this command lane is used to observe the order of variables in the dataframe for changing the keyword easier.

,# Lane Points,% Observed,Lane 1 Truck VHT (Veh-Hours),Lane 1 Truck VMT (Veh-Miles),Lane 2 Truck VHT (Veh-Hours),Lane 2 Truck VMT (Veh-Miles),Lane 3 Truck VHT (Veh-Hours),Lane 3 Truck VMT (Veh-Miles),Lane 4 Truck VHT (Veh-Hours),Lane 4 Truck VMT (Veh-Miles),Truck VHT (Veh-Hours),Truck VMT (Veh-Miles)
count,24179.0,24179.0,24179.0,24179.0,24179.0,24179.0,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000
mean,4.0,0.0,0.0,0.0,0.0,0.0,0.021428,2.179164,0.067546,3.497763,0.098540,5.676926
std,0.0,0.0,0.0,0.0,0.0,0.0,0.043962,0.939627,0.054798,1.535632,0.055621,2.450331
min,4.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.600000,0.000000,2.400000,0.100000,4.000000
50%,4.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.400000,0.100000,3.600000,0.100000,5.600000
75%,4.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.800000,0.100000,4.800000,0.100000,7.600000
max,4.0,0.0,0.0,0.0,0.0,0.0,0.200000,6.000000,0.400000,9.200000,0.600000,15.200000


In [30]:
data_do_fs.describe()

,# Lane Points,% Observed,Flow (Veh/5 Minutes),Lane 1 Flow (Veh/5 Minutes),Lane 1 Speed (mph),Lane 2 Flow (Veh/5 Minutes),Lane 2 Speed (mph),Lane 3 Flow (Veh/5 Minutes),Lane 3 Speed (mph),Lane 4 Flow (Veh/5 Minutes),Lane 4 Speed (mph),Speed (mph)
count,24179.0,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000
mean,4.0,37.325779,210.015013,42.331445,70.860660,68.196782,59.316936,59.121345,53.635676,40.365441,53.885835,58.861305
std,0.0,21.750741,100.731992,24.086096,6.995159,33.878660,19.666965,24.602199,18.801419,21.765148,17.796821,15.536899
min,4.0,0.000000,10.000000,0.000000,23.100000,0.000000,4.400000,3.000000,3.000000,0.000000,3.000000,9.200000
25%,4.0,0.000000,113.000000,20.000000,71.000000,36.000000,62.500000,36.000000,55.500000,20.000000,58.900000,62.100000
50%,4.0,50.000000,234.000000,45.000000,73.400000,76.000000,68.800000,66.000000,62.000000,42.000000,62.300000,66.000000
75%,4.0,50.000000,288.000000,60.000000,73.600000,97.000000,70.700000,79.000000,63.800000,57.000000,62.800000,67.300000
max,4.0,50.000000,437.000000,120.000000,82.300000,146.000000,78.100000,113.000000,74.100000,106.000000,72.800000,75.900000


In [31]:
# Caution: use dataframe.head() to observe the name and the position of every column
data_up = data_up_fs
data_up = data_up.iloc[:,2:13]
data_up.columns = ['t','f','f1','s1','f2','s2','f3','s3','f4','s4','s']
data_up[['o1','o2','o3','o4']] = data_up_o.iloc[:,[3,4,5,6]]
data_up[['tf1','tp1','tf2','tp2','tf3','tp3','tf4','tp4']] = data_up_tftp.iloc[:,[3,4,5,6,7,8,9,10]]
data_up[['vht1','vmt1','vht2','vmt2','vht3','vmt3','vht4','vmt4']] = data_up_vmtvht.iloc[:,[3,4,5,6,7,8,9,10]]
# if data_up and data_do have different numbers of raws, here may be an erro
# in November, upstream lacks the record of 11/12/2017 6:05, while dostream has.
data_up[['s_do1','s_do2','s_do3','s_do4']] = data_do_fs.iloc[:,[5,7,9,11]]
data_up = data_up.reset_index(drop=True)

In [32]:
data_up.describe()

,f,f1,s1,f2,s2,f3,s3,f4,s4,s,...,vht2,vmt2,vht3,vmt3,vht4,vmt4,s_do1,s_do2,s_do3,s_do4
count,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,...,24179.0,24179.0,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000,24179.000000
mean,234.748873,58.134662,73.573729,70.771620,71.124046,57.323214,64.709624,48.519376,58.289950,67.484515,...,0.0,0.0,0.021428,2.179164,0.067546,3.497763,70.860660,59.316936,53.635676,53.885835
std,112.637315,32.966940,10.063914,30.671316,10.529549,26.375251,9.110977,23.418149,7.991977,8.999222,...,0.0,0.0,0.043962,0.939627,0.054798,1.535632,6.995159,19.666965,18.801419,17.796821
min,10.000000,2.000000,16.400000,3.000000,15.600000,3.000000,14.500000,2.000000,16.900000,15.800000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,23.100000,4.400000,3.000000,3.000000
25%,141.000000,31.000000,73.600000,44.000000,69.900000,36.000000,63.700000,30.000000,56.900000,67.900000,...,0.0,0.0,0.000000,1.600000,0.000000,2.400000,71.000000,62.500000,55.500000,58.900000
50%,245.000000,59.000000,75.900000,75.000000,72.800000,59.000000,68.700000,49.000000,62.200000,70.400000,...,0.0,0.0,0.000000,2.400000,0.100000,3.600000,73.400000,68.800000,62.000000,62.300000
75%,303.000000,77.000000,76.500000,91.000000,73.700000,73.000000,69.000000,61.000000,62.800000,71.200000,...,0.0,0.0,0.000000,2.800000,0.100000,4.800000,73.600000,70.700000,63.800000,62.800000
max,684.000000,195.000000,85.500000,185.000000,88.000000,158.000000,75.500000,146.000000,69.100000,76.400000,...,0.0,0.0,0.200000,6.000000,0.400000,9.200000,82.300000,78.100000,74.100000,72.800000


###  1.2 Delete Outlier

In [33]:
data_up['t'] = pd.to_datetime(data_up['t'])
data = pd.DataFrame()
for month in range(12):
    
    for day in range(7): # There are 7 days data in each month
        # Select the data in the same day
        begin = data_up['t'][0] + datetime.timedelta(days=day) + relativedelta(months=month)
        end = data_up['t'][0] + datetime.timedelta(days=day+1) + relativedelta(months=month)
        interval = (data_up['t']>begin) & (data_up['t']<end)
        data_up_t = data_up[interval]

        # Computing Interquantile Range (IQR)
        upper_q_s1 = data_up_t['s1'].quantile(0.75)
        lower_q_s1 = data_up_t['s1'].quantile(0.25)
        IQR_s1 = upper_q_s1 - lower_q_s1
        upper_q_s2 = data_up_t['s2'].quantile(0.75)
        lower_q_s2 = data_up_t['s2'].quantile(0.25)
        IQR_s2 = upper_q_s2 - lower_q_s2
        upper_q_s3 = data_up_t['s3'].quantile(0.75)
        lower_q_s3 = data_up_t['s3'].quantile(0.25)
        IQR_s3 = upper_q_s3 - lower_q_s3
        upper_q_s4 = data_up_t['s4'].quantile(0.75)
        lower_q_s4 = data_up_t['s4'].quantile(0.25)
        IQR_s4 = upper_q_s4 - lower_q_s4
        
        # Delecting the records which are outlier in s1,s2,s3 and s4
        data_up_t = data_up_t[((lower_q_s1-1.5*IQR_s1) <= data_up_t['s1'])& (data_up_t['s1'] <= (upper_q_s1+1.5*IQR_s1))]
        data_up_t = data_up_t[((lower_q_s2-1.5*IQR_s2) <= data_up_t['s2'])& (data_up_t['s2'] <= (upper_q_s2+1.5*IQR_s2))]
        data_up_t = data_up_t[((lower_q_s3-1.5*IQR_s3) <= data_up_t['s3'])& (data_up_t['s3'] <= (upper_q_s3+1.5*IQR_s3))]
        data_up_t = data_up_t[((lower_q_s4-1.5*IQR_s4) <= data_up_t['s4'])& (data_up_t['s4'] <= (upper_q_s4+1.5*IQR_s4))]

        data = data.append(data_up_t, sort = True)
        
data_up = data
data_up.describe()

,f,f1,f2,f3,f4,o1,o2,o3,o4,s,...,tp3,tp4,vht1,vht2,vht3,vht4,vmt1,vmt2,vmt3,vmt4
count,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,...,22138.000000,22138.000000,22138.0,22138.0,22138.000000,22138.000000,22138.0,22138.0,22138.000000,22138.000000
mean,229.077514,56.339055,69.090297,56.098518,47.549643,6.102764,6.831584,7.088667,7.067097,67.419062,...,10.066971,19.338770,0.0,0.0,0.019401,0.063976,0.0,0.0,2.127419,3.403704
std,113.006913,32.953339,30.743256,26.541854,23.577487,7.788909,7.583283,7.827191,7.745792,8.568494,...,4.273362,8.236991,0.0,0.0,0.041539,0.053517,0.0,0.0,0.939523,1.529434
min,23.000000,3.000000,7.000000,6.000000,5.000000,0.200000,0.400000,0.400000,0.400000,15.800000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
25%,132.000000,28.000000,42.000000,34.000000,29.000000,1.800000,2.600000,2.800000,2.800000,67.600000,...,6.900000,10.900000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.600000,2.400000
50%,239.000000,57.000000,73.000000,58.000000,48.000000,3.700000,4.700000,4.900000,4.900000,70.300000,...,10.300000,21.400000,0.0,0.0,0.000000,0.100000,0.0,0.0,2.000000,3.200000
75%,297.000000,75.000000,90.000000,72.000000,60.000000,5.600000,6.500000,6.800000,6.600000,71.000000,...,12.100000,24.300000,0.0,0.0,0.000000,0.100000,0.0,0.0,2.800000,4.800000
max,684.000000,195.000000,185.000000,158.000000,146.000000,55.400000,56.700000,59.100000,60.500000,76.400000,...,27.300000,50.000000,0.0,0.0,0.200000,0.300000,0.0,0.0,6.000000,9.200000


### 1.3 Construct Variables

In [34]:
# Delecting the records whose f1 = 0, f2 = 0 or f3 = 0
data_up = data_up[(data_up['f1']!=0) & (data_up['f2']!=0) & (data_up['f3']!=0) & (data_up['f4']!=0)]
data_up.describe()

,f,f1,f2,f3,f4,o1,o2,o3,o4,s,...,tp3,tp4,vht1,vht2,vht3,vht4,vmt1,vmt2,vmt3,vmt4
count,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,...,22138.000000,22138.000000,22138.0,22138.0,22138.000000,22138.000000,22138.0,22138.0,22138.000000,22138.000000
mean,229.077514,56.339055,69.090297,56.098518,47.549643,6.102764,6.831584,7.088667,7.067097,67.419062,...,10.066971,19.338770,0.0,0.0,0.019401,0.063976,0.0,0.0,2.127419,3.403704
std,113.006913,32.953339,30.743256,26.541854,23.577487,7.788909,7.583283,7.827191,7.745792,8.568494,...,4.273362,8.236991,0.0,0.0,0.041539,0.053517,0.0,0.0,0.939523,1.529434
min,23.000000,3.000000,7.000000,6.000000,5.000000,0.200000,0.400000,0.400000,0.400000,15.800000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
25%,132.000000,28.000000,42.000000,34.000000,29.000000,1.800000,2.600000,2.800000,2.800000,67.600000,...,6.900000,10.900000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.600000,2.400000
50%,239.000000,57.000000,73.000000,58.000000,48.000000,3.700000,4.700000,4.900000,4.900000,70.300000,...,10.300000,21.400000,0.0,0.0,0.000000,0.100000,0.0,0.0,2.000000,3.200000
75%,297.000000,75.000000,90.000000,72.000000,60.000000,5.600000,6.500000,6.800000,6.600000,71.000000,...,12.100000,24.300000,0.0,0.0,0.000000,0.100000,0.0,0.0,2.800000,4.800000
max,684.000000,195.000000,185.000000,158.000000,146.000000,55.400000,56.700000,59.100000,60.500000,76.400000,...,27.300000,50.000000,0.0,0.0,0.200000,0.300000,0.0,0.0,6.000000,9.200000


In [35]:
# Spliting the data into three dataframes for lane1, lane2, lane3 and lane4
lane1list = ['t']+[col for col in data_up.columns if '1' in col]
data1 = data_up[lane1list]
lane2list = ['t']+[col for col in data_up.columns if '2' in col]
data2 = data_up[lane2list]
lane3list = ['t']+[col for col in data_up.columns if '3' in col]
data3 = data_up[lane3list]
lane4list = ['t']+[col for col in data_up.columns if '4' in col]
data4 = data_up[lane4list]
data1.head()

,t,f1,o1,s1,s_do1,tf1,tp1,vht1,vmt1
1,2017-01-08 00:05:00,26,1.5,75.9,73.6,0,0,0,0
2,2017-01-08 00:10:00,28,1.6,76.0,73.5,0,0,0,0
3,2017-01-08 00:15:00,25,1.4,75.9,73.6,0,0,0,0
4,2017-01-08 00:20:00,31,1.7,75.9,73.6,0,0,0,0
5,2017-01-08 00:25:00,22,1.4,75.9,73.6,0,0,0,0


In [36]:
# Change the key of every dataframe for iteration later.
variables = ['t','f','o','s','s_do','tf','tp','vht','vmt']
data1.columns = variables
data2.columns = variables
data3.columns = variables
data4.columns = variables
data1.loc[:,'s2'] = data2.loc[:,'s']
# Because lane2 is adjacent to lane1 and lane3 at the same time
data2.loc[:,'s1'] = data1.loc[:,'s']
data2.loc[:,'s3'] = data3.loc[:,'s']
data3.loc[:,'s2'] = data2.loc[:,'s']
data3.loc[:,'s4'] = data4.loc[:,'s']
data4.loc[:,'s3'] = data3.loc[:,'s']

data2.head()

E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,t,f,o,s,s_do,tf,tp,vht,vmt,s1,s3
1,2017-01-08 00:05:00,41,2.3,72.9,70.3,0,0,0,0,75.9,69.0
2,2017-01-08 00:10:00,43,2.4,73.0,69.7,0,0,0,0,76.0,69.1
3,2017-01-08 00:15:00,41,2.2,72.9,70.0,0,0,0,0,75.9,69.1
4,2017-01-08 00:20:00,46,2.4,72.8,70.8,0,0,0,0,75.9,69.0
5,2017-01-08 00:25:00,38,2.2,72.9,70.3,0,0,0,0,75.9,69.0


In [37]:
# use dummy variablesS
for lanedata in [data1, data2, data3, data4]:
    # ln(speed)
    lanedata.loc[:,'lns'] = lanedata['s'].map(lambda x: math.log(x))
    # truck speed
    lanedata.loc[:,'ts'] = 999
    lanedata.loc[lanedata['vht']!=0, 'ts'] =  lanedata['vmt']/lanedata['vht']
#     lanedata['ts'] = lanedata.apply(lambda x: 999.0 if x['vht'] == 0 else x['vmt']/x['vht'], axis = 1) 
#     '''axis : {0 or ‘index’, 1 or ‘columns’}, default 0
#         Axis along which the function is applied:
#         0 or ‘index’: apply function to each column.
#         1 or ‘columns’: apply function to each row.'''
    # traffic flow indicator and high truck flow indicator
    lanedata.loc[:,'fi'] = 0
    lanedata.loc[:,'htfi'] = 0
    lanedata.loc[lanedata['f']<75, 'fi'] = 1
    lanedata.loc[lanedata['tf']>100, 'htfi'] = 1
    # relative truck percentage indicator
    lanedata.loc[:,'tpia'] = 0
    lanedata.loc[:,'tpib'] = 0
    lanedata.loc[(lanedata['tp']>0.6)&(lanedata['f']<50), 'tpia'] = 1
    lanedata.loc[(lanedata['tp']<=0.2)&(lanedata['f']>200), 'tpib'] = 1
    # seasonal indicator (fall will be reference)
    lanedata.loc[:,'seasona'] = 0
    lanedata.loc[:,'seasonb'] = 0
    lanedata.loc[:,'seasonc'] = 0
    lanedata.loc[(2<lanedata['t'].dt.month) & (lanedata['t'].dt.month<6), 'seasona'] = 1 # Spring
    lanedata.loc[(5<lanedata['t'].dt.month) & (lanedata['t'].dt.month<9), 'seasonb'] = 1
    lanedata.loc[(8<lanedata['t'].dt.month) & (lanedata['t'].dt.month<12), 'seasonc'] = 1
    # time-of-week indicator
    lanedata.loc[:,'weeka'] = 0
    lanedata.loc[:,'weekb'] = 0
    lanedata.loc[:,'weekc'] = 0
    lanedata.loc[:,'weekd'] = 0
    lanedata.loc[:,'weeke'] = 0
    lanedata.loc[:,'weekf'] = 0
    lanedata.loc[lanedata['t'].dt.weekday==0, 'weeka'] = 1 # Monday
    lanedata.loc[lanedata['t'].dt.weekday==1, 'weekb'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==2, 'weekc'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==3, 'weekd'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==4, 'weeke'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==5, 'weekf'] = 1
    # time-of-day indicator
    lanedata['hour'] = lanedata['t'].dt.hour
    lanedata.loc[:,'toda'] = 0
    lanedata.loc[:,'todb'] = 0
    lanedata.loc[:,'todc'] = 0
    lanedata.loc[:,'todd'] = 0
    lanedata.loc[(0<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<6), 'toda'] = 1 # midnight to 6 am
    lanedata.loc[(7<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<9), 'todb'] = 1 # am peak
    lanedata.loc[(17<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<19), 'todc'] = 1 # pm peak
    lanedata.loc[(19<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<24), 'todd'] = 1 # night time
    
# ratio of flows in current lane to adjacent lanes
data1.loc[:,'fratio'] = data1['f']/data2['f']
# Use the average of the flow ratios with lane1 and lane3 as the ultimate flow ratio in lane2
# data2.loc[:,'fratio'] = (data2['f']/data1['f']+data2['f']/data3['f']) 
# data2.loc[:,'fratio3'] = data2['f']/data3['f']
data4.loc[:,'fratio'] = data4['f']/data3['f']

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
''' This block is used to determine the speed of lane2 is influenced by which lane (lan1 or lane3),
and the criterion is the flow ratio between lane 2 and lane1 and the flow ratio between lane 2 and lane3.
Bigger one will be dorminating!'''
data2.loc[:,'fratio1'] = data2['f']/data1['f']
data2.loc[:,'fratio3'] = data2['f']/data3['f']
data2.loc[:,'flag'] = 0
data2.loc[data2['fratio1']<data2['fratio3'], 'flag'] = 1 # using '<' recieves better result?
data2.loc[:,'fratio'] = 0
data2.loc[data2['flag']==1,'fratio'] = data2['fratio1']
data2.loc[data2['flag']==0,'fratio'] = data2['fratio3']
# consrtuting the instruments and endogeneity dataframe for lane2
data2_instru = data1[['t','o','f','ts','fi','s_do','s']]
data2_instru.loc[data2['flag']==0,['o','f','ts','fi','s_do','s']] = data3.loc[data2['flag']==0,['o','f','ts','fi','s_do','s']]

data2_instru.loc[:,'fratio'] = 0
data2_instru.loc[:,'fratio'] = data2.loc[:,'fratio']

data3.loc[:,'fratio2'] = data3['f']/data2['f']
data3.loc[:,'fratio4'] = data3['f']/data4['f']
data3.loc[:,'flag'] = 0
data3.loc[data3['fratio2']<data3['fratio4'], 'flag'] = 1 # using '<' recieves better result?
data3.loc[:,'fratio'] = 0
data3.loc[data3['flag']==1,'fratio'] = data3['fratio2']
data3.loc[data3['flag']==0,'fratio'] = data3['fratio4']
# consrtuting the instruments and endogeneity dataframe for lane2
data3_instru = data2[['t','o','f','ts','fi','s_do','s']]
data3_instru.loc[data3['flag']==0,['o','f','ts','fi','s_do','s']] = data4.loc[data3['flag']==0,['o','f','ts','fi','s_do','s']]

data3_instru.loc[:,'fratio'] = 0
data3_instru.loc[:,'fratio'] = data3.loc[:,'fratio']

In [39]:
data3_instru.describe()

,o,f,ts,fi,s_do,s,fratio
count,22138.000000,22138.000000,22138.0,22138.000000,22138.000000,22138.000000,22138.000000
mean,6.831584,69.090297,999.0,0.516759,61.419374,70.697498,0.805438
std,7.583283,30.743256,0.0,0.499730,17.913280,9.824809,0.063387
min,0.400000,7.000000,999.0,0.000000,4.400000,15.600000,0.647059
25%,2.600000,42.000000,999.0,0.000000,64.600000,69.600000,0.763889
50%,4.700000,73.000000,999.0,1.000000,69.300000,72.800000,0.779817
75%,6.500000,90.000000,999.0,1.000000,70.800000,73.400000,0.875000
max,56.700000,185.000000,999.0,1.000000,78.100000,88.000000,1.000000


In [40]:
data1[['o','f','ts','fi','fratio','s_do','s']].describe()

,o,f,ts,fi,fratio,s_do,s
count,22138.000000,22138.000000,22138.0,22138.000000,22138.000000,22138.000000,22138.000000
mean,6.102764,56.339055,999.0,0.740356,0.759143,71.179677,73.338427
std,7.788909,32.953339,0.0,0.438449,0.155710,6.937264,9.478608
min,0.200000,3.000000,999.0,0.000000,0.176471,23.100000,16.400000
25%,1.800000,28.000000,999.0,0.000000,0.685393,71.200000,73.300000
50%,3.700000,57.000000,999.0,1.000000,0.794118,73.500000,75.900000
75%,5.600000,75.000000,999.0,1.000000,0.860465,73.600000,76.300000
max,55.400000,195.000000,999.0,1.000000,1.059783,82.300000,85.500000


In [41]:
# data1['fratio'] = data1['f']/data2['f']
# data1 = data1.dropna()
data1.dtypes

t          datetime64[ns]
f                   int64
o                 float64
s                 float64
s_do              float64
tf                  int64
tp                  int64
vht                 int64
vmt                 int64
s2                float64
lns               float64
ts                float64
fi                  int64
htfi                int64
tpia                int64
tpib                int64
seasona             int64
seasonb             int64
seasonc             int64
weeka               int64
weekb               int64
weekc               int64
weekd               int64
weeke               int64
weekf               int64
hour                int64
toda                int64
todb                int64
todc                int64
todd                int64
fratio            float64
dtype: object

## 2 Prediction 

### 2.1 Error Examination

In [42]:
# data1[['f','fratio']] = data1[['f','fratio']].infer_objects()
# data2[['f','fratio']] = data2[['f','fratio']].infer_objects()
''' Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.
And the error come from other_var.
And after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'
are (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object 
to int and float, respectively.
'''

" Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.\nAnd the error come from other_var.\nAnd after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'\nare (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object \nto int and float, respectively.\n"

In [43]:
np.linalg.matrix_rank(data1[['o','f','ts','fi','fratio']])

5

In [44]:
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### 2.2 Predicting Used New Method (including downstream speed)

In [45]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_new = pd.DataFrame(columns=['MAE1','MAE2','MAE3','MAE4'])
speed_new1 = pd.DataFrame()
speed_new2 = pd.DataFrame()
speed_new3 = pd.DataFrame()
speed_new4 = pd.DataFrame()
for month in range(12):
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    data3_test = data3[data3['t'].dt.month == month+1]
    data3_train = data3[data3['t'].dt.month != month+1]
    data4_test = data4[data4['t'].dt.month == month+1]
    data4_train = data4[data4['t'].dt.month != month+1]
    
    data2_instru_test = data2_instru[data2_instru['t'].dt.month == month+1]
    data2_instru_train = data2_instru[data2_instru['t'].dt.month != month+1]
    data3_instru_test = data3_instru[data3_instru['t'].dt.month == month+1]
    data3_instru_train = data3_instru[data3_instru['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_train.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_train.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
#     exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var+['s_do1']],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments+['s_do2']]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var+['s_do2']],
         'endog': data2_instru_train[['s']],
         'instruments': data2_instru_train[instruments+['s_do']]}
    
    u3 = {'dependent': data3_train[['s3']],
         'exog': data3_train[other_var+['s_do3']],
         'endog': data3_instru_train[['s']],
         'instruments': data3_instru_train[instruments+['s_do']]}
    
    u4 = {'dependent': data4_train[['s4']],
         'exog': data4_train[other_var+['s_do4']],
         'endog': data4_train[['s3']],
         'instruments': data3_train[instruments+['s_do3']]}
    
    equations = dict(u1=u1, u2=u2, u3=u3, u4=u4)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_test.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_test.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
    
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var+['s_do1']],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments+['s_do2']]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var+['s_do2']],
         'endog': data2_instru_test[['s']],
         'instruments': data2_instru_test[instruments+['s_do']]}
    
    u3_test = {'dependent': data3_test[['s3']],
         'exog': data3_test[other_var+['s_do3']],
         'endog': data3_instru_test[['s']],
         'instruments': data3_instru_test[instruments+['s_do']]}
    
    u4_test = {'dependent': data4_test[['s4']],
         'exog': data4_test[other_var+['s_do4']],
         'endog': data4_test[['s3']],
         'instruments': data3_test[instruments+['s_do3']]}
    
    equations_test = dict(u1=u1_test, u2=u2_test, u3=u3_test, u4=u4_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    y_true3 = data3_test['s3']
    y_pred3 = system_3sls_pre['u3']
    y_true4 = data4_test['s4']
    y_pred4 = system_3sls_pre['u4']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_new.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_new.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    error_result_new.loc[month,'MAE3'] = mae(y_true3, y_pred3)
    error_result_new.loc[month,'MAE4'] = mae(y_true4, y_pred4)
    speed_new1 = pd.concat([speed_new1,y_pred1])
    speed_new2 = pd.concat([speed_new2,y_pred2])
    speed_new3 = pd.concat([speed_new3,y_pred3])
    speed_new4 = pd.concat([speed_new4,y_pred4])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # this command results in a result of a month only.

E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas

### 2.3 Predicting Used Old Method (not including downstream speed)

In [46]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_old = pd.DataFrame(columns=['MAE1','MAE2','MAE3','MAE4'])
speed_old1 = pd.DataFrame()
speed_old2 = pd.DataFrame()
speed_old3 = pd.DataFrame()
speed_old4 = pd.DataFrame()
for month in range(12):
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    data3_test = data3[data3['t'].dt.month == month+1]
    data3_train = data3[data3['t'].dt.month != month+1]
    data4_test = data4[data4['t'].dt.month == month+1]
    data4_train = data4[data4['t'].dt.month != month+1]
    
    data2_instru_test = data2_instru[data2_instru['t'].dt.month == month+1]
    data2_instru_train = data2_instru[data2_instru['t'].dt.month != month+1]
    data3_instru_test = data3_instru[data3_instru['t'].dt.month == month+1]
    data3_instru_train = data3_instru[data3_instru['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_train.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_train.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
#     exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var],
         'endog': data2_instru_train[['s']],
         'instruments': data2_instru_train[instruments]}
    
    u3 = {'dependent': data3_train[['s3']],
         'exog': data3_train[other_var],
         'endog': data3_instru_train[['s']],
         'instruments': data3_instru_train[instruments]}
    
    u4 = {'dependent': data4_train[['s4']],
         'exog': data4_train[other_var],
         'endog': data4_train[['s3']],
         'instruments': data3_train[instruments]}
    
    equations = dict(u1=u1, u2=u2, u3=u3, u4=u4)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_test.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_test.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
    
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var],
         'endog': data2_instru_test[['s']],
         'instruments': data2_instru_test[instruments]}
    
    u3_test = {'dependent': data3_test[['s3']],
         'exog': data3_test[other_var],
         'endog': data3_instru_test[['s']],
         'instruments': data3_instru_test[instruments]}
    
    u4_test = {'dependent': data4_test[['s4']],
         'exog': data4_test[other_var],
         'endog': data4_test[['s3']],
         'instruments': data3_test[instruments]}
    
    equations_test = dict(u1=u1_test, u2=u2_test, u3=u3_test, u4=u4_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    y_true3 = data3_test['s3']
    y_pred3 = system_3sls_pre['u3']
    y_true4 = data4_test['s4']
    y_pred4 = system_3sls_pre['u4']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_old.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_old.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    error_result_old.loc[month,'MAE3'] = mae(y_true3, y_pred3)
    error_result_old.loc[month,'MAE4'] = mae(y_true4, y_pred4)
    speed_old1 = pd.concat([speed_old1,y_pred1])
    speed_old2 = pd.concat([speed_old2,y_pred2])
    speed_old3 = pd.concat([speed_old3,y_pred3])
    speed_old4 = pd.concat([speed_old4,y_pred4])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # this command results in a result of a month only.

E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas

### 2.4 Result Exporting

In [47]:
# speed_new1.columns = ['new1']
# speed_new2.columns = ['new2']
# speed_new3.columns = ['new3']
# speed_new4.columns = ['new4']
# speed_old1.columns = ['old1']
# speed_old2.columns = ['old2']
# speed_old3.columns = ['old3']
# speed_old4.columns = ['old4']

In [48]:
speed = speed_new1
speed['new2'] = speed_new2
speed['new3'] = speed_new3
speed['new4'] = speed_new4
speed['old1'] = speed_old1
speed['old2'] = speed_old2
speed['old3'] = speed_old3
speed['old4'] = speed_old4
speed['observed1'] = data1['s']
speed['observed2'] = data2['s']
speed['observed3'] = data3['s']
speed['observed4'] = data4['s']
speed.rename(columns={0: 'new1'}, inplace=True)
speed.describe()

,new1,new2,new3,new4,old1,old2,old3,old4,observed1,observed2,observed3,observed4
count,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000,22138.000000
mean,72.805388,70.918228,63.840353,58.528248,71.404577,71.708185,58.828973,55.531372,73.338427,70.697498,64.888305,58.809802
std,8.724063,11.054540,10.008820,7.855665,9.624256,10.606872,15.264362,9.232433,9.478608,9.824809,8.752145,7.556931
min,16.826109,15.864922,8.592983,13.998174,23.615783,11.961131,1.662989,25.605650,16.400000,15.600000,14.500000,16.900000
25%,71.305841,66.076794,60.001325,57.550894,67.355423,67.941215,48.180530,49.537244,73.300000,69.600000,63.900000,57.800000
50%,74.854280,71.176759,66.134968,61.461806,72.316661,73.033934,64.834887,56.970825,75.900000,72.800000,68.800000,62.300000
75%,77.129421,75.904770,71.106672,62.666462,78.342595,76.878097,69.965877,62.087215,76.300000,73.400000,69.000000,62.800000
max,83.429574,98.452971,76.422465,67.468424,89.745632,92.338319,80.423965,69.633553,85.500000,88.000000,75.500000,65.500000


In [49]:
error_result = pd.DataFrame()
error_result['MAE_new1'] = error_result_new['MAE1']
error_result['MAE_new2'] = error_result_new['MAE2']
error_result['MAE_new3'] = error_result_new['MAE3']
error_result['MAE_new4'] = error_result_new['MAE4']
error_result['MAE_old1'] = error_result_old['MAE1']
error_result['MAE_old2'] = error_result_old['MAE2']
error_result['MAE_old3'] = error_result_old['MAE3']
error_result['MAE_old4'] = error_result_old['MAE4']
error_result

,MAE_new1,MAE_new2,MAE_new3,MAE_new4,MAE_old1,MAE_old2,MAE_old3,MAE_old4
0,2.00573,3.91457,4.91836,0.663487,6.78735,4.95828,21.7534,11.2209
1,2.68159,5.44797,7.62067,0.867444,6.53874,5.65477,20.9042,9.92729
2,1.55601,3.73397,4.91122,0.880354,2.651,2.73763,8.31791,5.64153
3,1.44247,3.38823,4.64566,0.714856,2.66206,2.76092,8.46466,5.29934
4,1.21042,2.65468,3.62784,0.576056,2.85444,2.97075,8.79781,5.2689
5,1.14019,2.90217,3.27751,0.626588,2.76889,2.79188,8.56415,5.25929
6,1.05412,2.75866,2.72847,0.577265,2.74038,2.88206,8.42663,5.04185
7,1.33574,3.34153,4.03424,0.682534,2.79741,2.79511,8.79309,5.6974
8,1.8374,3.90912,2.76123,1.21135,2.80764,3.54148,9.73829,4.33492
9,3.90403,3.41327,3.91401,2.15877,3.54532,3.49785,10.3707,5.33283


In [50]:
speed.to_excel('G:\\Modeling Endogeneity\\Results\\400m_4lanes_speed.xlsx')
error_result.to_excel('G:\\Modeling Endogeneity\\Results\\400m_4lanes_error.xlsx')
data_up[['t','f1','f2','f3','f4']].to_excel('G:\\Modeling Endogeneity\\Results\\400m_4lanes_flow.xlsx')